In [1]:
from __future__ import division
def feature_eng(y, fs, vis=False):
    
    import librosa
    import librosa.display
    import numpy as np
    import maad
    from soundsig.sound import BioSound
    import matplotlib.pyplot as plt
    import pandas as pd
    import warnings
    warnings.simplefilter('ignore')
    
    # spectrum
    ps = np.abs(np.fft.fft(y))**2
    time_step = 1/fs
    freqs = np.fft.fftfreq(y.size, time_step)
    ps = ps[0:int((len(ps)/2)-1)] # take out the negative freq
    freqs = freqs[0:int((len(freqs)/2)-1)]

    # melspectrogram
    S = librosa.feature.melspectrogram(y=y, sr=fs)
    S_dB = librosa.power_to_db(S, ref=np.max)
    
    # alpha indices
    Sxx_power,tn,fn,ext = maad.sound.spectrogram (y, fs, mode='psd')
    df_temporal_indices = maad.features.all_temporal_alpha_indices(y, fs)
    df_spectral_indices, _ = maad.features.all_spectral_alpha_indices(Sxx_power,tn,fn, extent=ext)
    df_indices = pd.concat([df_temporal_indices,df_spectral_indices], axis=1)
    
    # soundsig for modulation power spectrum
    
    myBioSound = BioSound(soundWave=y, fs=fs)
    myBioSound.mpsCalc(window=1, Norm = True)
    
    # reduce the dimension of MPS to one quadrant
    len1 = int(len(myBioSound.wf-1)/2)
    len2 = int(len(myBioSound.wt-1)/2)
    quad1 = myBioSound.mps[len1:,len2:]
    quad2 = np.fliplr(myBioSound.mps[len1:,:len2+1])
    mps = (quad1+quad2)/2
    wf = myBioSound.wf[len1:]
    wt = myBioSound.wt[len2:]
    
    if vis==True:
        plt.figure()
        plt.plot(np.arange(0,y.size/fs,1/fs), y)
        plt.show()
        
        fig, ax = plt.subplots()
        img = librosa.display.specshow(S_dB, x_axis='time',
                                 y_axis='mel', sr=fs,
                                 fmax=fs/2, ax=ax)
        fig.colorbar(img, ax=ax, format='%+2.0f dB')
        ax.set(title='Mel-frequency spectrogram')
        
        DBNOISE=100
        plt.figure()
        plt.clf()
        cmap = plt.get_cmap('jet')
        ex = (myBioSound.wt.min(), myBioSound.wt.max(), myBioSound.wf.min()*1e3, myBioSound.wf.max()*1e3)
        logMPS = 10.0*np.log10(myBioSound.mps)
        maxMPS = logMPS.max()
        minMPS = maxMPS-DBNOISE
        logMPS[logMPS < minMPS] = minMPS
        plt.imshow(logMPS, interpolation='nearest', aspect='auto', origin='lower', cmap=cmap, extent=ex)
        plt.ylabel('Spectral Frequency (Cycles/KHz)')
        plt.xlabel('Temporal Frequency (Hz)')
        plt.colorbar()
        plt.ylim((0,myBioSound.wf.max()*1e3))
        plt.title('Modulation Power Spectrum')
        plt.show()
        
    
    return ps, freqs, S_dB, df_indices, mps, wt, wf
    

In [2]:
def preproc(file, fs=16000, vis=False):
    import librosa
    import numpy as np
    import matplotlib.pyplot as plt
    import noisereduce as nr
    
    raw_y, fs = librosa.load(file, sr=fs, duration=10, mono = True)
    y_mono_rs = raw_y - np.mean(raw_y) # remove DC
    rms = np.sqrt(np.mean(y_mono_rs**2)) # get rms
    y = y_mono_rs/(rms/0.1) # normalize the rms to 0.1
    
    fg_y = nr.reduce_noise(y=y, sr=fs)
    bg_y = y - fg_y
    
    
    if vis == True: print('++++++++++++++++++++++++ raw ++++++++++++++++++++++++')
    ps, freqs, S_dB, df_indices, mps, wt, wf = feature_eng(y, fs, vis)
    if vis == True: print('++++++++++++++++++++ foreground ++++++++++++++++++++')
    ps_fg, freqs_fg, S_dB_fg, df_indices_fg, mps_fg, wt_fg, wf_fg = feature_eng(fg_y, fs, vis)
    if vis == True: print('++++++++++++++++++++ background ++++++++++++++++++++')
    ps_bg, freqs_fg, S_dB_bg, df_indices_bg, mps_bg, wt_bg, wf_bg = feature_eng(bg_y, fs, vis)
    
    output = {'fs': fs, 'y': y, 'fg_y': fg_y, 'bg_y': bg_y,
             'df_indices': df_indices, 'mps': mps, 'wt': wt, 'wf': wf, 
             'df_indices_fg': df_indices_fg, 'mps_fg': mps_fg,
             'df_indices_bg': df_indices_bg, 'mps_bg': mps_bg}
    
    return output
    

In [3]:
# scan data directories
import glob
import pandas as pd

nature_file_list = []
nature_file_list += glob.glob('../data/raw/AmbisonicSoundLibrary/nature/*')
nature_file_list += glob.glob('../data/raw/GoogleAudioSet/Outside, rural or natural/*')
nature_file_list += glob.glob('../data/raw/youtube/NatureSoundscapes/*')
nature_file_list += glob.glob('../data/raw/youtube/NomadicAmbience_nature/*')
nature_file_list += glob.glob('../data/raw/S2L_LULC/non_urban/*')
nature_file_list += glob.glob('../data/raw/S2L_LULC/urban_0_25/*')

city_file_list = []
city_file_list += glob.glob('../data/raw/GoogleAudioSet/Outside, urban or manmade/*')
city_file_list += glob.glob('../data/raw/youtube/NomadicAmbience_city/*')
city_file_list += glob.glob('../data/raw/S2L_LULC/urban_26_100/*')

In [4]:
# as there are too many files in SONYC, so only a few are sampled
from random import seed, sample
seed(23)
SONYC_file_list = sample(glob.glob('../data/raw/SONYC/**/*.wav', recursive=True),800)
city_file_list += SONYC_file_list

pd.DataFrame({'nature_file_list': nature_file_list}).to_csv('../data/raw/nature_file_list.csv')
pd.DataFrame({'city_file_list': city_file_list}).to_csv('../data/raw/city_file_list.csv')

In [5]:
# testing cell
# preproc(city_file_list[0])

In [ ]:
import time
import pickle
import os

from joblib import Parallel, delayed

def run_preproc(file_name):
    save_file_name = '../data/interim/'+file_name[12:-4]+'.pkl'
    if not os.path.isfile(save_file_name): # run the script only if the file does not exist
        if not os.path.exists(save_file_name.rsplit('/', 1)[0]): # create the folder if the folder does not exist
            os.makedirs(save_file_name.rsplit('/', 1)[0]) # extract the folder of the file path
            

        start_time = time.time()

        output = preproc(file_name, vis = False)
        f = open(save_file_name,'wb') # create a binary pickle file 
        pickle.dump(output,f)
        f.close()

        print("--- %s seconds ---" % (time.time() - start_time))
        

# run the process in parallel
Parallel(n_jobs=-1)(delayed(run_preproc)(file_name) for file_name in nature_file_list+city_file_list)

In [7]:
# len(nature_file_list)+len(city_file_list)

In [8]:
# for file_name in nature_file_list+city_file_list:
#     print(file_name)
#     run_preproc(file_name)

--- 12.155380010604858 seconds ---
--- 7.5888378620147705 seconds ---
--- 7.953967094421387 seconds ---
--- 7.760721921920776 seconds ---
--- 7.4243597984313965 seconds ---
--- 7.927980899810791 seconds ---
--- 7.430912017822266 seconds ---
--- 7.381556034088135 seconds ---
--- 7.124932050704956 seconds ---
--- 7.330318927764893 seconds ---
--- 7.681488037109375 seconds ---
--- 7.287230014801025 seconds ---
--- 7.086978912353516 seconds ---
--- 7.385751962661743 seconds ---
--- 7.391186237335205 seconds ---
--- 7.47281813621521 seconds ---
--- 7.361771106719971 seconds ---
--- 7.20461893081665 seconds ---
--- 7.203245162963867 seconds ---
--- 8.65666389465332 seconds ---
--- 7.552986145019531 seconds ---
--- 7.336986064910889 seconds ---
--- 7.246613025665283 seconds ---
--- 7.630435943603516 seconds ---
--- 7.956050872802734 seconds ---
--- 7.773560047149658 seconds ---
--- 7.282544851303101 seconds ---
--- 7.413444995880127 seconds ---
--- 8.309169292449951 seconds ---
--- 8.67993712

--- 12.808779001235962 seconds ---
--- 9.556394100189209 seconds ---
--- 9.942615032196045 seconds ---
--- 8.773659229278564 seconds ---
--- 10.325793027877808 seconds ---
--- 11.35223388671875 seconds ---
--- 9.129035949707031 seconds ---
--- 10.634848833084106 seconds ---
--- 9.503909826278687 seconds ---
--- 9.255484104156494 seconds ---
--- 9.407426834106445 seconds ---
--- 10.094305992126465 seconds ---
--- 9.441053867340088 seconds ---
--- 11.903187036514282 seconds ---
--- 10.714482069015503 seconds ---
--- 10.12264895439148 seconds ---
--- 9.61695671081543 seconds ---
--- 9.718766212463379 seconds ---
--- 9.977257251739502 seconds ---
--- 9.968318939208984 seconds ---
--- 9.943718910217285 seconds ---
--- 9.579989194869995 seconds ---
--- 7.791393041610718 seconds ---
--- 7.7117838859558105 seconds ---
--- 7.931987047195435 seconds ---
--- 8.180809259414673 seconds ---
--- 11.075525999069214 seconds ---
--- 12.43633770942688 seconds ---
--- 11.611583948135376 seconds ---
--- 10

--- 12.407742977142334 seconds ---
--- 7.978610038757324 seconds ---
--- 7.716421127319336 seconds ---
--- 7.247102975845337 seconds ---
--- 7.283530950546265 seconds ---
--- 8.046369075775146 seconds ---
--- 8.15320110321045 seconds ---
--- 7.947251796722412 seconds ---
--- 7.562771797180176 seconds ---
--- 7.185198783874512 seconds ---
--- 8.11854100227356 seconds ---
--- 7.803346872329712 seconds ---
--- 7.516004800796509 seconds ---
--- 8.082469940185547 seconds ---
--- 8.338608264923096 seconds ---
--- 10.567121744155884 seconds ---
--- 10.036202907562256 seconds ---
--- 10.893920183181763 seconds ---
--- 10.699061870574951 seconds ---
--- 11.026562213897705 seconds ---
--- 10.622200012207031 seconds ---
--- 10.095829963684082 seconds ---
--- 10.267696142196655 seconds ---
--- 10.134662866592407 seconds ---
--- 10.544342994689941 seconds ---
--- 10.463369846343994 seconds ---
--- 10.436490058898926 seconds ---
--- 10.363416194915771 seconds ---
--- 10.434420108795166 seconds ---
-

/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


--- 11.749226093292236 seconds ---
--- 7.344854116439819 seconds ---
--- 7.527780055999756 seconds ---
--- 7.281693935394287 seconds ---
--- 7.779819011688232 seconds ---
--- 8.793330192565918 seconds ---
--- 8.233196020126343 seconds ---
--- 8.783944129943848 seconds ---
--- 9.923517942428589 seconds ---
--- 8.43952202796936 seconds ---
--- 8.512504816055298 seconds ---
--- 8.42152714729309 seconds ---
--- 8.939706802368164 seconds ---
--- 10.039621829986572 seconds ---
--- 9.571261882781982 seconds ---
--- 9.943739891052246 seconds ---
--- 8.477171897888184 seconds ---
--- 9.118264198303223 seconds ---
--- 9.558046340942383 seconds ---
--- 8.164369106292725 seconds ---
--- 9.902778148651123 seconds ---
--- 9.76642107963562 seconds ---
--- 7.964730978012085 seconds ---
--- 8.06468677520752 seconds ---
--- 8.319301843643188 seconds ---
--- 8.262342929840088 seconds ---
--- 8.229604959487915 seconds ---
--- 8.851464986801147 seconds ---
--- 9.524581909179688 seconds ---
--- 8.9702360630

--- 13.751069068908691 seconds ---
--- 8.126161813735962 seconds ---
--- 9.940305709838867 seconds ---
--- 8.265903949737549 seconds ---
--- 8.253348112106323 seconds ---
--- 8.448711156845093 seconds ---
--- 10.309280157089233 seconds ---
--- 8.373418092727661 seconds ---
--- 8.79434585571289 seconds ---
--- 8.324012041091919 seconds ---
--- 9.693915128707886 seconds ---
--- 9.005253791809082 seconds ---
--- 8.739949703216553 seconds ---
--- 8.172538757324219 seconds ---
--- 7.898154020309448 seconds ---
--- 9.822566747665405 seconds ---
--- 9.549532890319824 seconds ---
--- 9.373840808868408 seconds ---
--- 8.409323930740356 seconds ---
--- 9.76471996307373 seconds ---
--- 8.154145956039429 seconds ---
--- 8.616389036178589 seconds ---
--- 8.488513946533203 seconds ---
--- 8.099753856658936 seconds ---
--- 7.911096096038818 seconds ---
--- 9.297177076339722 seconds ---
--- 8.87437891960144 seconds ---
--- 8.606498003005981 seconds ---
--- 10.91036581993103 seconds ---
--- 11.15635204